# libraries importation

In [26]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import pathlib
from tensorflow.keras.utils import image_dataset_from_directory
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import resnet50

## data importation

In [28]:
waste_nc = pathlib.Path('/home/fa-mtl/code/Loupy2023/get_it_right/raw_data/waste_nc/TRAIN.1') 
garb_bio = list(waste_nc.glob('B/*'))
garb_nbio = list(waste_nc.glob('N/*'))

img = Image.open(str(garb_bio[15]))
img_height, img_width =  img.size

In [29]:
data = image_dataset_from_directory(waste_nc)
class_names = data.class_names
print(class_names)

Found 59922 files belonging to 2 classes.
['B', 'N']


In [30]:
img_width
img.size

(200, 200)

# Data split

In [31]:
batch_size = 16
img_height = 200
img_width = 200

In [48]:
dataset_url = waste_nc

In [49]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  dataset_url,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 59922 files belonging to 2 classes.
Using 47938 files for training.


In [50]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  dataset_url,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size )

Found 59922 files belonging to 2 classes.
Using 11984 files for validation.


# Model


In [51]:
# define the pre-processing layers
shape=(img_height, img_width, 3)

# Define the model architecture
def initialize_model():
    model = Sequential([
      layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
      layers.Conv2D(16, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(32, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Flatten(),
      layers.Dense(32, activation='relu'),
      layers.Dense(2)
    ])
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model

In [52]:
model = initialize_model()

In [53]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_2 (Rescaling)     (None, 200, 200, 3)       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 200, 200, 16)      448       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 100, 100, 16)     0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 100, 100, 32)      4640      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 50, 50, 32)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 80000)            

In [54]:
#Train the model
epochs=10
history = model.fit(
  train_ds,
  validation_data=test_ds,
  epochs=1
)

 228/2997 [=>............................] - ETA: 35:58 - loss: 0.4458 - accuracy: 0.8268